# CVS Health Community Access Analysis - Part 4: Advanced Visualizations

This notebook creates detailed statistical visualizations including box plots, histograms, and bar charts to better understand patterns in the data.


In [ ]:
# import libraries for visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# load data (assumes you've run notebooks 06a, 06b, and 06c)
# if running independently, load and prepare data first
print("ready for advanced visualizations")


## Setup: Ensure Required Columns Exist

we need to make sure all required columns exist before creating visualizations. this makes the notebook more robust.


In [ ]:
# ensure required columns exist - create them if they don't
# this makes the notebook more robust if cells are run out of order

# create health_burden_score if it doesn't exist
if 'health_burden_score' not in df.columns:
    health_vars = ['stroke', 'physical_inactivity', 'self_care_disability', 'social_isolation']
    available_vars = [var for var in health_vars if var in df.columns]
    if available_vars:
        df['health_burden_score'] = df[available_vars].mean(axis=1)
        print(f"created health_burden_score from: {available_vars}")
    else:
        print("warning: could not create health_burden_score - required columns missing")

# ensure population column exists (needed for some visualizations)
if 'population' not in df.columns:
    print("warning: 'population' column not found. some visualizations may be unavailable.")
    # create a dummy population column to prevent errors
    df['population'] = 100000  # default value

# create clinics_per_100k if it doesn't exist
if 'clinics_per_100k' not in df.columns:
    if 'population' in df.columns and 'clinic_count' in df.columns:
        df['clinics_per_100k'] = (df['clinic_count'] / df['population']) * 100000
        # replace inf values with 0
        df['clinics_per_100k'] = df['clinics_per_100k'].replace([np.inf, -np.inf], 0)
        print("created clinics_per_100k column")
    else:
        print("warning: cannot create clinics_per_100k - missing required columns")

# create health_need and gap_score if they don't exist
if 'health_need' not in df.columns and 'health_burden_score' in df.columns:
    df['health_need'] = (
        (df['health_burden_score'] - df['health_burden_score'].min()) /
        (df['health_burden_score'].max() - df['health_burden_score'].min())
    )
    print("created health_need column")

if 'gap_score' not in df.columns and 'health_need' in df.columns and 'clinic_count' in df.columns:
    if df['clinic_count'].max() > 0:
        df['clinic_availability'] = df['clinic_count'] / df['clinic_count'].max()
    else:
        df['clinic_availability'] = 0
    df['gap_score'] = df['health_need'] - df['clinic_availability']
    print("created gap_score column")

if 'pop_adjusted_gap' not in df.columns and 'health_need' in df.columns and 'population' in df.columns:
    df['pop_adjusted_gap'] = df['health_need'] * df['population']
    print("created pop_adjusted_gap column")

print("\n✓ all required columns checked/created. ready for visualizations!")


## Box Plot 1: Clinic Count Distribution by SVI Quartiles

this box plot shows how clinic counts vary across different levels of social vulnerability. we divide counties into quartiles based on their SVI scores to see if more vulnerable counties have fewer clinics.


In [ ]:
# create SVI quartiles for comparison
# quartiles divide counties into 4 equal groups based on SVI scores
df['svi_quartile'] = pd.qcut(df['svi_overall'], q=4, labels=['Low SVI (Q1)', 'Medium-Low SVI (Q2)', 
                                                              'Medium-High SVI (Q3)', 'High SVI (Q4)'])

# create box plot to show distribution of clinic counts by SVI quartile
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='svi_quartile', y='clinic_count', palette='viridis')
plt.title('Distribution of CVS Clinic Counts by Social Vulnerability Index Quartiles', 
          fontsize=14, fontweight='bold')
plt.xlabel('SVI Quartile', fontsize=12)
plt.ylabel('Number of Clinics', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# print summary statistics
print("\nsummary statistics by SVI quartile:")
print(df.groupby('svi_quartile')['clinic_count'].describe())


### what this box plot shows:

- the box shows the interquartile range (middle 50% of counties)
- the line in the box is the median (middle value)
- the whiskers extend to show the range of values
- outliers are shown as individual points
- if boxes are lower for high SVI quartiles, it means vulnerable counties have fewer clinics
- the median is likely zero for all quartiles, showing most counties have no clinics
- but the upper quartiles and outliers reveal where clinics are concentrated


## Box Plot 2: Health Burden Score by Clinic Presence

this box plot compares health burden scores across counties with different levels of clinic access. this helps us see if counties without clinics have worse health outcomes.


In [ ]:
# create clinic presence categories
# we group counties by how many clinics they have
df['clinic_presence'] = df['clinic_count'].apply(lambda x: 'No Clinics' if x == 0 else 
                                                  ('1-2 Clinics' if x <= 2 else '3+ Clinics'))

# create box plot comparing health burden by clinic presence
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='clinic_presence', y='health_burden_score', palette='Set2', 
           order=['No Clinics', '1-2 Clinics', '3+ Clinics'])
plt.title('Health Burden Score Distribution by CVS Clinic Presence', fontsize=14, fontweight='bold')
plt.xlabel('Clinic Presence Category', fontsize=12)
plt.ylabel('Health Burden Score', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# print statistical summary
print("\nhealth burden by clinic presence:")
print(df.groupby('clinic_presence')['health_burden_score'].agg(['mean', 'median', 'std']))


### what this box plot shows:

- if the "No Clinics" box is higher than others, it means counties without clinics have worse health
- the median line shows the typical health burden for each group
- wider boxes indicate more variation in health burden within that group
- this visualization directly shows the relationship between access and health outcomes
- if there's a clear pattern, it suggests clinics might be avoiding high-need areas, or high-need areas lack clinics


## Histogram 1: Distribution of Clinic Counts

histograms show the frequency distribution of clinic counts. this helps us understand how clinics are distributed across all counties.


In [ ]:
# create two histograms side by side
# one shows all counties, one zooms in on counties with 0-10 clinics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# histogram of all clinic counts
axes[0].hist(df['clinic_count'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_title('Distribution of Clinic Counts (All Counties)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Number of Clinics', fontsize=11)
axes[0].set_ylabel('Number of Counties', fontsize=11)
axes[0].axvline(df['clinic_count'].median(), color='red', linestyle='--', linewidth=2, 
                label=f'Median: {df["clinic_count"].median():.1f}')
axes[0].legend()
axes[0].grid(alpha=0.3)

# histogram focusing on counties with 0-10 clinics (zoomed in view)
axes[1].hist(df[df['clinic_count'] <= 10]['clinic_count'], bins=11, edgecolor='black', 
            alpha=0.7, color='coral')
axes[1].set_title('Distribution of Clinic Counts (0-10 Clinics)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Number of Clinics', fontsize=11)
axes[1].set_ylabel('Number of Counties', fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# print key statistics
print(f"\nkey statistics:")
print(f"counties with 0 clinics: {(df['clinic_count'] == 0).sum()} ({(df['clinic_count'] == 0).mean()*100:.1f}%)")
print(f"counties with 1+ clinics: {(df['clinic_count'] > 0).sum()} ({(df['clinic_count'] > 0).mean()*100:.1f}%)")
print(f"counties with 3+ clinics: {(df['clinic_count'] >= 3).sum()} ({(df['clinic_count'] >= 3).mean()*100:.1f}%)")


### what these histograms show:

- the left histogram shows the full distribution - most bars are at zero (no clinics)
- the red dashed line shows the median, which is likely zero
- the right histogram zooms in to see detail for counties with few clinics
- the height of each bar shows how many counties have that number of clinics
- if most bars are at zero, it means most counties have no clinics
- this reveals the highly skewed nature of clinic distribution


## Histogram 2: Distribution of Health Burden and SVI Scores

these histograms show how health burden and social vulnerability are distributed across counties. this helps us understand the range and typical values of these important metrics.


In [ ]:
# create two histograms side by side for health burden and SVI
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# health burden score distribution
axes[0].hist(df['health_burden_score'], bins=50, edgecolor='black', alpha=0.7, color='purple')
axes[0].set_title('Distribution of Health Burden Scores', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Health Burden Score', fontsize=11)
axes[0].set_ylabel('Number of Counties', fontsize=11)
axes[0].axvline(df['health_burden_score'].mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {df["health_burden_score"].mean():.2f}')
axes[0].axvline(df['health_burden_score'].median(), color='orange', linestyle='--', linewidth=2, 
                label=f'Median: {df["health_burden_score"].median():.2f}')
axes[0].legend()
axes[0].grid(alpha=0.3)

# SVI overall distribution
axes[1].hist(df['svi_overall'], bins=50, edgecolor='black', alpha=0.7, color='teal')
axes[1].set_title('Distribution of Social Vulnerability Index (SVI)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('SVI Score', fontsize=11)
axes[1].set_ylabel('Number of Counties', fontsize=11)
axes[1].axvline(df['svi_overall'].mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {df["svi_overall"].mean():.3f}')
axes[1].axvline(df['svi_overall'].median(), color='orange', linestyle='--', linewidth=2, 
                label=f'Median: {df["svi_overall"].median():.3f}')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


### what these histograms show:

- the shape of the distribution tells us if values are normally distributed, skewed, or have other patterns
- the mean (red line) is the average value across all counties
- the median (orange line) is the middle value - half of counties are above, half below
- if mean and median are close, the distribution is roughly symmetric
- if they're far apart, the distribution is skewed (has a tail in one direction)
- these distributions help us understand what "typical" values look like for comparison


## Histogram 3: Clinics per 100k Population Distribution

this histogram shows the distribution of clinic density (clinics per 100,000 people). this accounts for population size, so we can fairly compare rural and urban counties.


In [ ]:
# filter out infinite values and very high outliers for better visualization
# we focus on the 99th percentile to see the main distribution
df_plot = df[df['clinics_per_100k'].notna() & (df['clinics_per_100k'] < df['clinics_per_100k'].quantile(0.99))]

plt.figure(figsize=(10, 6))
plt.hist(df_plot['clinics_per_100k'], bins=50, edgecolor='black', alpha=0.7, color='darkgreen')
plt.title('Distribution of Clinics per 100,000 Population', fontsize=14, fontweight='bold')
plt.xlabel('Clinics per 100,000 People', fontsize=12)
plt.ylabel('Number of Counties', fontsize=12)
plt.axvline(df_plot['clinics_per_100k'].median(), color='red', linestyle='--', linewidth=2, 
            label=f'Median: {df_plot["clinics_per_100k"].median():.2f}')
plt.axvline(df_plot['clinics_per_100k'].mean(), color='orange', linestyle='--', linewidth=2, 
            label=f'Mean: {df_plot["clinics_per_100k"].mean():.2f}')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nclinics per 100k statistics:")
print(f"median: {df_plot['clinics_per_100k'].median():.2f}")
print(f"mean: {df_plot['clinics_per_100k'].mean():.2f}")
print(f"counties with 0 clinics per 100k: {(df['clinics_per_100k'] == 0).sum()} ({(df['clinics_per_100k'] == 0).mean()*100:.1f}%)")


### what this histogram shows:

- this metric adjusts for population size, so it's fair to compare large and small counties
- most counties likely have zero clinics per 100k (left side of histogram)
- the distribution shows how clinic density varies across counties
- counties with higher values have better clinic access relative to their population
- this helps identify counties where clinic density is low despite having enough people to support one


## Bar Chart 1: Top 20 Most Underserved Counties (Population-Adjusted)

this bar chart shows the counties with the highest combination of health need and population size but low clinic access. these are the highest-priority expansion targets.


In [ ]:
# get top 20 underserved counties based on population-adjusted gap
# this metric combines health need with population size to prioritize high-impact areas
top_underserved = df.nlargest(20, 'pop_adjusted_gap')[
    ['county_full', 'state_full', 'pop_adjusted_gap', 'health_burden_score', 'clinic_count', 'population']]

# create horizontal bar chart
# color code by clinic count: red = 0, orange = 1-2, green = 3+
plt.figure(figsize=(12, 8))
colors = ['#d62728' if x == 0 else '#ff7f0e' if x <= 2 else '#2ca02c' 
         for x in top_underserved['clinic_count']]
bars = plt.barh(range(len(top_underserved)), top_underserved['pop_adjusted_gap'], color=colors)
plt.yticks(range(len(top_underserved)), 
           [f"{row['county_full']}, {row['state_full']}" for _, row in top_underserved.iterrows()])
plt.xlabel('Population-Adjusted Gap Score', fontsize=12, fontweight='bold')
plt.title('Top 20 Most Underserved Counties (High Need × Population)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()  # show highest at top
plt.grid(axis='x', alpha=0.3)

# add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#d62728', label='0 Clinics'),
                   Patch(facecolor='#ff7f0e', label='1-2 Clinics'),
                   Patch(facecolor='#2ca02c', label='3+ Clinics')]
plt.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

print("\ntop 20 underserved counties details:")
print(top_underserved.to_string())


### what this bar chart shows:

- longer bars indicate higher priority for expansion (higher need × larger population)
- red bars are counties with zero clinics - highest priority
- orange bars have 1-2 clinics - may need additional clinics
- green bars have 3+ clinics but still show high need - may indicate very large populations
- this chart directly identifies where CVS can make the biggest impact
- counties at the top of the chart should be prioritized for expansion


## Bar Chart 2: State-Level Clinic Coverage Summary

this bar chart shows which states have the highest clinic density (clinics per 100,000 people). this helps identify states where CVS has strong presence and states with opportunities.


In [ ]:
# calculate state-level summary statistics
state_stats = df.groupby('state_full').agg({
    'clinic_count': 'sum',
    'population': 'sum',
    'health_burden_score': 'mean',
    'svi_overall': 'mean'
}).reset_index()

# calculate clinics per 100k for each state
state_stats['clinics_per_100k'] = (state_stats['clinic_count'] / state_stats['population']) * 100000
state_stats = state_stats.sort_values('clinics_per_100k', ascending=False).head(15)

# create horizontal bar chart
plt.figure(figsize=(12, 8))
bars = plt.barh(range(len(state_stats)), state_stats['clinics_per_100k'], color='steelblue')
plt.yticks(range(len(state_stats)), state_stats['state_full'])
plt.xlabel('Clinics per 100,000 Population', fontsize=12, fontweight='bold')
plt.title('Top 15 States by CVS Clinic Density (Clinics per 100k)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# add value labels on bars
for i, (idx, row) in enumerate(state_stats.iterrows()):
    plt.text(row['clinics_per_100k'] + 0.1, i, f"{row['clinics_per_100k']:.2f}", 
             va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\ntop 15 states by clinic density:")
print(state_stats[['state_full', 'clinic_count', 'population', 'clinics_per_100k']].to_string(index=False))


### what this bar chart shows:

- states with longer bars have better clinic coverage relative to their population
- this helps identify where CVS has strong market presence
- states not in the top 15 may have expansion opportunities
- comparing this to health burden data can reveal if well-served states have lower health needs
- states with low clinic density but high health burden are priority expansion targets


## Bar Chart 3: States with Highest Underserved Population

this bar chart shows which states have the most people living in underserved counties (high health need but no clinics). this helps prioritize states for expansion.


In [ ]:
# calculate underserved population by state
# underserved = counties with high health need (top 25%) but zero clinics
high_need_no_clinic = df[(df['health_burden_score'] >= df['health_burden_score'].quantile(0.75)) & 
                         (df['clinic_count'] == 0)]

state_underserved = high_need_no_clinic.groupby('state_full').agg({
    'population': 'sum',
    'county_full': 'count'
}).reset_index()
state_underserved.columns = ['state_full', 'underserved_population', 'underserved_counties']
state_underserved = state_underserved.sort_values('underserved_population', ascending=False).head(15)

# create horizontal bar chart
plt.figure(figsize=(12, 8))
bars = plt.barh(range(len(state_underserved)), state_underserved['underserved_population'] / 1e6, 
                color='crimson')
plt.yticks(range(len(state_underserved)), state_underserved['state_full'])
plt.xlabel('Underserved Population (Millions)', fontsize=12, fontweight='bold')
plt.title('Top 15 States with Highest Underserved Population\n(High Health Need + Zero Clinics)', 
          fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# add value labels showing population and county count
for i, (idx, row) in enumerate(state_underserved.iterrows()):
    plt.text(row['underserved_population']/1e6 + 0.05, i, 
             f"{row['underserved_population']/1e6:.2f}M\n({int(row['underserved_counties'])} counties)", 
             va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\ntop 15 states with underserved populations:")
print(state_underserved.to_string(index=False))


### what this bar chart shows:

- longer bars indicate states with more people living in underserved counties
- the numbers show both total population (in millions) and number of underserved counties
- states at the top have the most people who could benefit from CVS expansion
- this helps prioritize which states to focus expansion efforts on
- states with many underserved counties but small populations may be less of a priority than states with fewer counties but larger populations
